# Name: Matthias Bartolo                            Id: 0436103L

# Web Intelligence Individual Assignment (Information Retrieval Task)

### Packages

In [1]:
import nltk #For tokenisation
import os   #For file retrieval
import xml.etree.cElementTree as et #For extracting xml data from a file
from nltk.tokenize import RegexpTokenizer #For tokenising and removing punctuation
from nltk.stem import PorterStemmer #For text Stemming
from nltk.corpus import stopwords #For stop word removal
nltk.download('stopwords')#Downloading stop words from nltk library
import math #For log
import numpy as np #For Document Matrix
from numpy.linalg import norm #For normalising cosine similarity
import pandas as pd #For the creation of a dataframe
from numpy import dot #For calculation of dot product

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Indexing Component

### 1. Parse the document to extract the data in the XML’s < raw > tag (10 marks);


In [2]:
#Function which retrieves,a list of files in the doc directory
def RetrievingFiles(directoryName):
    #Retrieves all the file names int he given directory
    FileList= os.listdir(directoryName)
    #Dictionary which holds the full path of the files
    Filepaths={}
    #Looping through all the file names in the file list
    for filename in FileList:
        #Attaching directoryName at the beginning of the file name to achieve full file path
        fullpath=os.path.join(directoryName, filename)
        #If full path is a directory, then we will opt to add all the files inside the directory to the
        #Filepaths list, this is achieved by recursively, calling the same function
        if os.path.isdir(fullpath):
            Filepaths.update(RetrievingFiles(fullpath))
        else:
            #Appending full path to list
            Filepaths[filename]=fullpath
    #Retuning List of full file paths
    return Filepaths

In [3]:
#Function which extracts the data in XML raw tag
def ExtractRawData(filepath):
    #Importing data by reading through file
    tree=et.parse(filepath)
    root=tree.getroot()
    #Variable which will hold the extractedData
    extractedData=[]
    #Extracts information which have the raw tag
    for rawData in root.iter("raw"):
        #Retrieving text and saving text in variable
        extractedData=rawData.text
    #Returning text
    return extractedData

### 2. Tokenise the documents’ content (5 marks);

In [4]:
#Function which tokenises the document's content and removes punctuation
def Tokenise(data):
    #Creating reference variable for class RegexpTokenizer, which is found in the nltk class
    tk=RegexpTokenizer(r'\w+')#\w+ is used to match any word character and thus punctuation would be ignored
    #Using tokenise method
    tokenisedDocument=tk.tokenize(data)
    #Returning tokenised document text
    return tokenisedDocument

### 3. Perform case-folding, stop-word removal and stemming (20 marks);


In [5]:
#Function, which handles case folding, i.e., returns string in list in lowercase
def CaseFolding(textList):
    #Creating and returning new list, whereby, every word is in lowercase
    filteredList=[word.casefold() for word in textList]
    return filteredList

In [6]:
#Function which removes stop words (common words) from the english language
def StopWordRemoval(textList):
    #List to hold text without stop words
    filteredList=[]
    #Retrieving stopwords
    stopWords=set(stopwords.words('english'))
    #Looping through all the words in the inputted list, and if they are not stop words, then
    #words are appended to the filtered list
    for word in textList:
        if word not in stopWords:
            filteredList.append(word)
    #Returning filtered List
    return filteredList

In [7]:
#Function which handles Stemming, i.e., removes suffixes and prefixes, from strings in list
def Stemming(textList):
    #Creating reference variable for class PorterStemmer
    stemmer=PorterStemmer()
    #Creating and returning new list, whereby, every word is stemmed
    filteredList =[stemmer.stem(word) for word in textList] 
    #Returning filtered List
    return filteredList

In [8]:
#Function that handles Case-folding, Stop-word removal and Stemming
#by calling all previous functions in the correct order
def DocumentCleaning(text):
    tokenisedList=Tokenise(text)
    casefoldedList=CaseFolding(tokenisedList)
    stopwordRemovedList=StopWordRemoval(casefoldedList)
    stemmedList=Stemming(stopwordRemovedList)
    return stemmedList

In [9]:
#Function, which calls all previous functions, and returns a filtered list of all the documents
def DocumentIndexing(directoryName):
    #Document list to hold all the text in all documents
    documentList={}
    #Retrieving all files from specified directory
    allFiles = RetrievingFiles(directoryName)
    #Loooping through every file
    for file in allFiles.keys():
        #Extracting data from the file
        data=ExtractRawData(allFiles[file])
        #Cleaning extractedData
        cleanedData=DocumentCleaning(data)
        #Appending cleaned data to document list
        documentList[file]=cleanedData
    #Returning documentList
    return documentList

### 4. Build the term by document matrix containing the T F.IDF weight for each term within each document (25 marks).

In [10]:
#Function which returns all the unique words in all of the documents
def GetUniqueWords(documentList):
    #Dictionary to hold the frequency of every word (utilised for fast indexing)
    wordFrequency={}
    
    #Looping through the text in the list, and if word is not present in wordFrequency dictionary,
    #then word will be added to dictionary with frequency of 1, else if word is present
    #frequency will be incremented    
    for text in documentList.keys():
        for word in documentList[text]:
            if word not in wordFrequency.keys():
                wordFrequency[word]=1
            else:
                wordFrequency[word]+=1
                
    #Returning list
    return wordFrequency

#### Calculating Term Frequency (TF) = 
(the normalised frequency of the term in the document ) / (the frequency of the most frequently occurring term in the document )

In [11]:
#Function which calculates the term frequency values of TF.IDF weight
def GetTermFrequency(WordFreqList,documentList):
    #Dictionary to hold the term frequency
    termfrequency={}
    
    #Looping through all the words in the WordFreqList.keys()
    for word in WordFreqList.keys():
        #Vector which holds, the WordTF for every document
        WordTFVector={}
        #Looping through all the documents in the documentList
        for document in documentList.keys():
            #Variable to act as a counter, to count the number of times, word appears in the document
            documentFrequency=0
            #Looping through all the document words in the document
            for documentWord in documentList[document]:
                #Incrementing documentFrequency since, word == documentWord
                if word == documentWord:
                    documentFrequency+=1
            #Appending documentFrequency to vector
            WordTFVector[document]=documentFrequency
        #Updating term frequency
        termfrequency[word]=WordTFVector
            
    #Normalising term frequency 
    #Looping through all the documents in the documentList
    for index in documentList.keys():
        #Getting the max Term Frequency for each document
        maxTf=0
        for word in termfrequency.keys():
            if(maxTf<termfrequency[word][index]):
                 maxTf= termfrequency[word][index]
        divisor=maxTf
        #Normalising by dividing by element with max term frequency
        for word in termfrequency.keys():
            termfrequency[word][index] = termfrequency[word][index]/divisor
            
    #Returning term frequency
    return termfrequency

#### Calculating Inverse Document Frequency (IDF) = 
log((the total number of documents) / (the number of documents containing the word))


In [12]:
#Function which calculates the inverse document frequency values of TF.IDF weight
def GetInverseDocumentFrequency(WordFreqList,documentList):
    #Dictionary to hold the inverse document frequency
    inverseDocumentfrequency={}
    
    #Looping through all the words in the WordFreqList.keys()
    for word in WordFreqList.keys():
        #Variable to act as a counter, to count the number of documents, word appears in
        documentAppearsCounter=0
        #Looping through all the documents in the documentList
        for document in documentList.keys():
            #If word is in document, then incrementing the counter
            if word in documentList[document]:
                documentAppearsCounter+=1
        #Calculating inverse document frequency by taking the log((no of documents)/(no of documents containing the word)) 
        inverseDocumentfrequency[word]= math.log((len(documentList.keys())/(documentAppearsCounter)),10) 
    #Returning inverse document frequency
    return inverseDocumentfrequency

#### Calculating TF.IDF
by multiplying TF X IDF

In [13]:
#Function which calculates the TF.IDF weight
def GetTFIDF(TFValues,IDFValues):
    #List which will hold the column names
    columnKeys=[]
    #Dictionary which will hold the TFIDFRows values
    TFIDFList={}
    #Looping through the keys in the TFValues dictionary
    for word in TFValues.keys():
        #Dictionary which will hold the TFIDFScore values of every row
        TFIDFRows={}
        #Looping through the elements in TFValues dictionary
        for wordTF in TFValues[word].keys():
            #Multiplying TF WITH IDF
            TFIDFScore= TFValues[word][wordTF]*IDFValues[word]
            #Appending TFIDF score to TFIDFRows
            TFIDFRows[wordTF]=TFIDFScore
            columnKeys=TFValues[word].keys()
        #Appending TFIDFRows to the TFIDFList
        TFIDFList[word]=TFIDFRows
        
    #Converting TFIDFList into a dataframe
    TFIDFMatrix=pd.DataFrame(TFIDFList,index=columnKeys, columns=TFIDFList.keys())
    #Returning TFIDFMatrix
    return TFIDFMatrix

#### Method which Calculates the Vector Space Model of a given list of documents

In [14]:
#Function that utilises all previously built methods to build the term by document matrix
def CalculateVectorSpaceModel(documentList):
    sortedWordFreqList=GetUniqueWords(documentList)
    TFValues=GetTermFrequency(sortedWordFreqList,documentList)
    IDFValues=GetInverseDocumentFrequency(sortedWordFreqList,documentList)
    documentMatrix=GetTFIDF(TFValues,IDFValues)
    return documentMatrix

In [15]:
directoryName1 = 'docs/'; #Directory which contains, all the doc files
documentList1=DocumentIndexing(directoryName1)
documentVectorSpaceModel1=CalculateVectorSpaceModel(documentList1)

In [16]:
#Printing Document Vector Space
documentVectorSpaceModel1

,william,beaumont,human,digest,physiolog,imag,sourc,novemb,21,1785,...,monterey,apporov,rio,gila,arcángel,247,assisi,asiacut,commanch,apach
wes2015.d001.naf,0.342024,2.519828,0.123613,1.596537,0.480216,0.058357,0.080294,0.066985,0.099801,0.130465,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d002.naf,0.000000,0.000000,0.067425,0.000000,0.000000,0.000000,0.000000,0.073075,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d003.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080382,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d004.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.053868,0.000000,0.123665,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d005.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036537,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wes2015.d327.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d328.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074851,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d329.naf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wes2015.d330.naf,0.273619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Querying Component

### 1. Preprocess the user query (tokenisation, case-folding, stop-word removal and stemming) (5 marks);

In [17]:
#User input of query
query=input("Enter user query:\n")
#Utilising above Methods
filteredquery=DocumentCleaning(query)
print(filteredquery)

Enter user query:
william beaumont
['william', 'beaumont']


In [18]:
#Queries already present in queriesfolder
directoryName2 = 'queries/'; #Directory which contains, all the query files
documentList2=DocumentIndexing(directoryName2)
print(documentList2)

{'wes2015.q01.naf': ['fabric', 'music', 'instrument'], 'wes2015.q02.naf': ['famou', 'german', 'poetri'], 'wes2015.q03.naf': ['romantic'], 'wes2015.q04.naf': ['univers', 'edinburgh', 'research'], 'wes2015.q06.naf': ['bridg', 'construct'], 'wes2015.q07.naf': ['walk', 'fame', 'star'], 'wes2015.q08.naf': ['scientist', 'work', 'atom', 'bomb'], 'wes2015.q09.naf': ['invent', 'internet'], 'wes2015.q10.naf': ['earli', 'telecommun', 'method'], 'wes2015.q12.naf': ['explor', 'south', 'pole'], 'wes2015.q13.naf': ['famou', 'member', 'royal', 'navi'], 'wes2015.q14.naf': ['nobel', 'prize', 'win', 'invent'], 'wes2015.q16.naf': ['south', 'america'], 'wes2015.q17.naf': ['edward', 'teller', 'mari', 'curi'], 'wes2015.q18.naf': ['comput', 'languag', 'program', 'artifici', 'intellig'], 'wes2015.q19.naf': ['william', 'hearst', 'movi'], 'wes2015.q22.naf': ['captain', 'jame', 'cook', 'becom', 'explor'], 'wes2015.q23.naf': ['grace', 'hopper', 'get', 'famou'], 'wes2015.q24.naf': ['comput', 'astronomi'], 'wes2015.

### 2. Use cosine similarity to calculate the similarity between the query and each document (25 marks);

#### Calculating Query Term Frequency (TF)
Utilising similar method to one above

In [19]:
#Function which calculates the query term frequency values of TF.IDF weight
def GetQueryTermFrequency(WordFreqList,queryList):
    #Dictionary to hold the query term frequency
    querytermfrequency={}
    #Looping through all the words in the WordFreqList.keys()
    for word in WordFreqList.keys():
        #Variable to act as a counter, to count the number of times, word appears in the query
        queryWordFrequency=0
        #Looping through all the words in the query list
        for queryWord in queryList:
            #If word is in query, then incrementing the counter
            if word == queryWord:
                queryWordFrequency+=1
        #Storing Frequency in dictionary
        querytermfrequency[word]= queryWordFrequency
    #Normalising:   
    maxTfIndex = max(querytermfrequency, key=querytermfrequency.get)
    divisor=querytermfrequency[maxTfIndex]
    for index in querytermfrequency.keys():
        querytermfrequency[index]=querytermfrequency[index]/divisor
        
    #Returning query term frequency
    return querytermfrequency

#### Calculating Query TF.IDF
Utilising similar method to one above

In [20]:
#Function which calculates the Query TF.IDF weight
def GetQueryTFIDF(TFValues,IDFValues):
    #List which will hold the TFIDFListvalues
    TFIDFList={}
    
    #Looping through the keys in the TFValues dictionary
    for word in TFValues.keys():
        TFIDFScore= TFValues[word]*IDFValues[word]
        #Appending TFIDFScore score to TFIDFList
        TFIDFList[word]=TFIDFScore
             
    return TFIDFList

#### Method which Calculates the QueryVector of a given query

In [21]:
#Function that utilises all previously built methods to build the query vector
def CalculateQueryVector(documentList,queryList):
    sortedWordFreqList=GetUniqueWords(documentList)
    TFValues=GetQueryTermFrequency(sortedWordFreqList,queryList)
    IDFValues=GetInverseDocumentFrequency(sortedWordFreqList,documentList)
    documentMatrix=GetQueryTFIDF(TFValues,IDFValues)
    return documentMatrix

#### Calculating Cosine Similarity = 
dotproduct(document * query) / (norm(document)*norm(query)

In [22]:
#Function which calculates, the similarity between the query and each document
def CosineSimilarity(documentVectorSpace,queryVector):
    #Dictionary to hold Cosine Similarity
    cosineSimilarity={}
    filenames=documentVectorSpace.index
    #Looping through document VectorSpace
    for i in filenames:
        #Calculating dot product
        dotproduct=dot(documentVectorSpace.loc[i],list( queryVector.values()))
        #Calculating Cosine Similarity
        cosSimilarity = dotproduct / (norm(documentVectorSpace.loc[i]) * norm(list(queryVector.values())))
        #Storing cosine Similarity in dictionary
        cosineSimilarity[i]=cosSimilarity
    #Returning dictionary
    return cosineSimilarity

#### Calculating Cosine Similarity upon user inputted query

In [23]:
userQueryVectorModel1=CalculateQueryVector(documentList1,filteredquery)
cosineSimilarity1=CosineSimilarity(documentVectorSpaceModel1,userQueryVectorModel1)

#### Calculating Cosine Similarity upon query present in queries folder

In [24]:
userQueryVectorModel2=CalculateQueryVector(documentList1,documentList2["wes2015.q01.naf"])
cosineSimilarity2=CosineSimilarity(documentVectorSpaceModel1,userQueryVectorModel2)

### 3.Output the list of documents as a ranked list (10 marks).

In [25]:
sortedCosine1={Key: Value for Key, Value in sorted(cosineSimilarity1.items(), key=lambda item: item[1], reverse=True)}
print("User query: '",query,"'\n")
for index in sortedCosine1.keys():
    print("Document:", index , "\t - ", "Cosine Similarity:\t ", round(sortedCosine1[index],4))

User query: ' william beaumont '

Document: wes2015.d001.naf 	 -  Cosine Similarity:	  0.659
Document: wes2015.d273.naf 	 -  Cosine Similarity:	  0.0411
Document: wes2015.d310.naf 	 -  Cosine Similarity:	  0.0287
Document: wes2015.d069.naf 	 -  Cosine Similarity:	  0.0269
Document: wes2015.d102.naf 	 -  Cosine Similarity:	  0.0269
Document: wes2015.d330.naf 	 -  Cosine Similarity:	  0.026
Document: wes2015.d136.naf 	 -  Cosine Similarity:	  0.0241
Document: wes2015.d320.naf 	 -  Cosine Similarity:	  0.0241
Document: wes2015.d028.naf 	 -  Cosine Similarity:	  0.024
Document: wes2015.d078.naf 	 -  Cosine Similarity:	  0.0228
Document: wes2015.d056.naf 	 -  Cosine Similarity:	  0.0204
Document: wes2015.d015.naf 	 -  Cosine Similarity:	  0.0189
Document: wes2015.d088.naf 	 -  Cosine Similarity:	  0.0174
Document: wes2015.d138.naf 	 -  Cosine Similarity:	  0.0127
Document: wes2015.d035.naf 	 -  Cosine Similarity:	  0.0125
Document: wes2015.d055.naf 	 -  Cosine Similarity:	  0.0125
Document:

In [26]:
sortedCosine2={Key: Value for Key, Value in sorted(cosineSimilarity2.items(), key=lambda item: item[1], reverse=True)}
print("Query from queries folder: '",documentList2["wes2015.q01.naf"],"'\n")
for index in sortedCosine2.keys():
    print("Document:", index , "\t - ", "Cosine Similarity:\t ", round(sortedCosine2[index],4))

Query from queries folder: ' ['fabric', 'music', 'instrument'] '

Document: wes2015.d016.naf 	 -  Cosine Similarity:	  0.1172
Document: wes2015.d085.naf 	 -  Cosine Similarity:	  0.0702
Document: wes2015.d259.naf 	 -  Cosine Similarity:	  0.0692
Document: wes2015.d254.naf 	 -  Cosine Similarity:	  0.067
Document: wes2015.d186.naf 	 -  Cosine Similarity:	  0.0512
Document: wes2015.d209.naf 	 -  Cosine Similarity:	  0.0479
Document: wes2015.d153.naf 	 -  Cosine Similarity:	  0.0353
Document: wes2015.d008.naf 	 -  Cosine Similarity:	  0.0337
Document: wes2015.d170.naf 	 -  Cosine Similarity:	  0.0311
Document: wes2015.d163.naf 	 -  Cosine Similarity:	  0.0291
Document: wes2015.d185.naf 	 -  Cosine Similarity:	  0.028
Document: wes2015.d215.naf 	 -  Cosine Similarity:	  0.0279
Document: wes2015.d154.naf 	 -  Cosine Similarity:	  0.0248
Document: wes2015.d315.naf 	 -  Cosine Similarity:	  0.0243
Document: wes2015.d089.naf 	 -  Cosine Similarity:	  0.0222
Document: wes2015.d296.naf 	 -  Cosi